**Import Libraries**

In [1]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models 
import torch.nn as nn
import numpy as np

**Check using cuda GPU**

In [2]:

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


**Load data and tarnsforms**

In [22]:

mean = ([0.4849, 0.4726, 0.4692])
std = ([0.2097, 0.2032, 0.2185])

transform = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(), transforms.Normalize(mean=mean, std=std)])

dataset = datasets.ImageFolder(root='./locations', transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

batch_size = 20
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)



**Mean and STD Calculation**

In [23]:

# mean = 0.0
# std = 0.0
# total_images_count = 0
# for images, _ in train_loader:
#     images = images.view(images.size(0), images.size(1), -1)
#     mean += images.mean(2).sum(0)
#     std += images.std(2).sum(0)
#     total_images_count += images.size(0)
# mean /= total_images_count
# std /= total_images_count
# print(mean)
# print(std)

# mean = ([0.4849, 0.4726, 0.4692])
# std = ([0.2097, 0.2032, 0.2185])


**Load Model**

In [24]:
# Load the model and move it to the GPU
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(in_features=model.fc.in_features, out_features=5)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


/home/abualmun/env/main/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/abualmun/env/main/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


**Start Training**

In [27]:

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):
        # Move images and labels to GPU
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        print(f'Epoch [{epoch+1} index:{i}   loss:{loss.item():.4f}')
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")


Epoch [1 index:0   loss:0.0086
Epoch [1 index:1   loss:0.6076
Epoch [1 index:2   loss:0.1010
Epoch [1 index:3   loss:0.2422
Epoch [1 index:4   loss:0.2515
Epoch [1 index:5   loss:0.0092
Epoch [1 index:6   loss:0.1210
Epoch [1 index:7   loss:0.0986
Epoch [1 index:8   loss:0.0986
Epoch [1 index:9   loss:0.0567
Epoch [1 index:10   loss:0.1683
Epoch [1 index:11   loss:0.1606
Epoch [1 index:12   loss:0.0571
Epoch [1 index:13   loss:0.0405
Epoch [1 index:14   loss:0.0453
Epoch [1 index:15   loss:0.3073
Epoch [1 index:16   loss:0.5758
Epoch [1 index:17   loss:0.6971
Epoch [1 index:18   loss:0.2174
Epoch [1 index:19   loss:0.3306
Epoch [1 index:20   loss:0.1006
Epoch [1 index:21   loss:0.0669
Epoch [1 index:22   loss:0.1795
Epoch [1 index:23   loss:0.0462
Epoch [1 index:24   loss:0.1690
Epoch [1 index:25   loss:0.0650
Epoch [1 index:26   loss:0.2524
Epoch [1 index:27   loss:0.0729
Epoch [1 index:28   loss:0.3888
Epoch [1 index:29   loss:0.2908
Epoch [1 index:30   loss:0.1223
Epoch [1 index:31 

**Validation**

In [28]:

# Validation (optional)
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in val_loader:
        # Move images and labels to GPU
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Validation Accuracy: {100 * correct / total:.2f}%')

Validation Accuracy: 84.02%


**Save the model's state dictionary**

In [29]:
torch.save(model.state_dict(), 'five_classes_model.pth')